In [45]:
using HDF5
using ITensors
using Plots
using Base.Threads
using Normalization
using Random, Distributions
using StatsBase
using QuadGK, Roots

In [2]:
function loadMPS(path::String; id::String="W")
    """Loads an MPS from a .h5 file. Returns and ITensor MPS."""
        file = path[end-2:end] != ".h5" ? path * ".h5" : path
        f = h5open("$file","r")
        mps = read(f, "$id", MPS)
        close(f)
        return mps
    end

loadMPS (generic function with 1 method)

In [103]:
loaded_mps = loadMPS("../saved/saved.h5");

In [104]:
function slice_mps_into_label_states(mps::MPS)
    """Gets the label index of the MPS and slices according to the number of classes (dim of the label index)"""
    """Assume one-hot encoding scheme i.e. class 0 = [1, 0], class 1 = [0, 1], etc. """
    dec_index = findindex(mps[end], "f(x)")
    if isnothing(dec_index)
        error("Label index not found on the first site of the MPS!")
    end
    # infer num classes from the dimension of the label index
    n_states = ITensors.dim(dec_index)
    states = []

    for i = 1:n_states
        # make a copy of the MPS so we are protected from any unintentional changes
        state = deepcopy(mps)
        if !isapprox(norm(state), 1.0) @warn "WARNING, MPS NOT NORMALISED!" end
        # create a onehot encoded tensor to slice the MPS
        decision_state = onehot(dec_index => (i))
        println("Class $(i-1) state: $(vector(decision_state))")
        # slice the mps along the dimension i by contracting with the label site
        state[end] *= decision_state

        # normalise the label MPS
        normalize!(state)
        push!(states, state)

    end

    return states

end

slice_mps_into_label_states (generic function with 1 method)

In [105]:
function probability_density(x::Float64, rdm::Matrix)
    """Function to compute the probability density for a given value, x ∈ [0, 1] according to
    the 1-site reduced density matrix (RDM)."""
    state = [exp(1im * (3π/2) * x) * cospi(0.5 * x), exp(-1im * (3π/2) * x) * sinpi(0.5 * x)] # our complex feature map
    return real(state' * rdm * state) # |<x|ρ|x>|
end

function get_normalisation_factor(rdm::Matrix)
    # get the normalisation factor for the rdm
    prob_density_wrapper(x) = probability_density(x, rdm)
    norm_factor, _ = quadgk(prob_density_wrapper, 0, 1)
    return norm_factor
end

function cdf(x, rdm, norm_factor)
    """Compute the cumulative distribution function"""
    prob_density_wrapper(x_prime) = probability_density(x_prime, rdm) / norm_factor
    cdf_value, _ = quadgk(prob_density_wrapper, 0, x)
    return cdf_value
end

function sample_individual_state(rdm, norm_factor)
    """Sample a state from the rdm using inverse transform sampling. Returns both the sampled value of
    x and the state after applying the feature map to the sample value for projective measurment."""
    u = rand()
    cdf_wrapper(x) = cdf(x, rdm, norm_factor) - u
    sampled_x = find_zero(cdf_wrapper, (0, 1))
    sampled_state = [exp(1im * (3π/2) * sampled_x) * cospi(0.5 * sampled_x), exp(-1im * (3π/2) * sampled_x) * sinpi(0.5 * sampled_x)]
    return sampled_x, sampled_state
end

function sample_mps(mps_original::MPS)
    """Revised version of the original sampling algorithm, fixed for continuous distribution"""
    mps = deepcopy(mps_original) # just in case ITensor does some funny business with in-place operations
    s = siteinds(mps)  
    x_samples = Vector{Float64}(undef, length(mps))
    for i in eachindex(mps)

        orthogonalize!(mps, i)
        # get the rdm 
        rdm = prime(mps[i], s[i]) * dag(mps[i])
        # check properties
        if !isapprox(real(tr(rdm)), 1.0; atol=1E-3) @warn "Trace of RDM ρ at site $i not equal to 1 ($(abs(tr(rdm))))." end
        if !isequal(rdm.tensor, adjoint(rdm).tensor) @warn "RDM at site $i not Hermitian." end
        rdm_m = matrix(rdm)
        # now sample from the rdm
        norm_factor = get_normalisation_factor(rdm_m)
        #println(norm_factor)
        sampled_x, sampled_state = sample_individual_state(rdm_m, norm_factor)
        x_samples[i] = sampled_x

        # construct projector
        sampled_state_as_ITensor = ITensor(sampled_state, s[i])
        m = MPS(1)
        m[1] = sampled_state_as_ITensor
        state_projector = projector(m)
        # make into a local MPO
        state_projector_operator = op(matrix(state_projector[1]), s[i])
        # apply to mps at site i
        mps[i] *= state_projector_operator
        # unprime indicies on updated site - indices get primed when applying MPO 
        noprime!(mps[i])
        normalize!(mps)
    end

    return x_samples
    
end

sample_mps (generic function with 1 method)

In [6]:
function feature_map(x::Float64)
    s1 = exp(1im * (3π/2) * x) * cospi(0.5 * x)
    s2 = exp(-1im * (3π/2) * x) * sinpi(0.5 * x)
    return [s1, s2]
end

feature_map (generic function with 1 method)

In [177]:
function generate_toy_timeseries(time_series_length::Int, total_dataset_size::Int, 
    train_split=0.7; random_state=1234, plot_examples=false)
    """Generate two sinusoids of different frequency, and with randomised phase.
    Inject noise with a given amplitude."""
    Random.seed!(random_state)

    train_size = floor(Int, total_dataset_size * train_split)
    test_size = total_dataset_size - train_size

    X_train = zeros(Float64, train_size, time_series_length)
    y_train = zeros(Int, train_size)
    
    X_test = zeros(Float64, test_size, time_series_length)
    y_test = zeros(Int, test_size)

    function generate_sinusoid(length::Int, A::Float64=1.0, 
        f::Float64=1.0, sigma=0.2)
        # sigma is scale of the gaussian noise added to the sinusoid
        t = range(0, 2π, length)
        phase = rand(Uniform(0, 2π)) # randomise the phase

        return A .* sin.(f .*t .+ phase) .+ sigma .* randn(length)

    end

    # generation parameters
    A1, f1, sigma1 = 1.0, 5.0, 0.0 # Class 0
    A2, f2, sigma2 = 1.0, 10.0, 0.0 # Class 1

    for i in 1:train_size
        label = rand(0:1) # choose a label, if 0 use freq f0, if 1 use freq f1. 
        data = label == 0 ? generate_sinusoid(time_series_length, A1, f1, sigma1) : 
            generate_sinusoid(time_series_length, A2, f2, sigma2)
        X_train[i, :] = data
        y_train[i] = label
    end

    for i in 1:test_size
        label = rand(0:1) # choose a label, if 0 use freq f0, if 1 use freq f1. 
        data = label == 0 ? generate_sinusoid(time_series_length, A1, f1, sigma1) : 
            generate_sinusoid(time_series_length, A2, f2, sigma2)
        X_test[i, :] = data
        y_test[i] = label
    end

    # plot some examples
    if plot_examples
        class_0_idxs = findall(x -> x.== 0, y_train)[1:2] # select subset of 5 samples
        class_1_idxs = findall(x -> x.== 1, y_train)[1:2]
        p0 = plot(X_train[class_0_idxs, :]', xlabel="Time", ylabel="x", title="Class 0 Samples (Unscaled)", 
            alpha=0.4, c=:red, label="")
        p1 = plot(X_train[class_1_idxs, :]', xlabel="Time", ylabel="x", title="Class 1 Samples (Unscaled)", 
            alpha=0.4, c=:magenta, label="")
        p = plot(p0, p1, size=(1200, 500), bottom_margin=5mm, left_margin=5mm)
        display(p)
    end

    return (X_train, y_train), (X_test, y_test)

end

generate_toy_timeseries (generic function with 2 methods)

In [178]:
(X_train, y_train), (X_test, y_test) = generate_toy_timeseries(100, 100);
rs = RobustSigmoid(X_train)
X_train_scaled = rs(X_train)

70×100 Matrix{Float64}:
 0.428252  0.363951  0.316105  0.28739   …  0.576028  0.502296  0.428252
 0.318661  0.432894  0.580586  0.688679     0.315227  0.278149  0.318661
 0.302811  0.279705  0.334183  0.459073     0.548534  0.402034  0.302811
 0.295756  0.331678  0.386128  0.455051     0.282217  0.279309  0.295756
 0.721802  0.715292  0.689351  0.644146     0.676692  0.708983  0.721802
 0.412614  0.307937  0.278716  0.327085  …  0.677383  0.560003  0.412614
 0.285317  0.311763  0.357457  0.420105     0.290644  0.278313  0.285317
 0.704672  0.717861  0.653528  0.522068     0.47066   0.614651  0.704672
 0.28036   0.299358  0.33781   0.394478     0.300489  0.280734  0.28036
 0.343228  0.473038  0.616628  0.705487     0.297345  0.281504  0.343228
 ⋮                                       ⋱                      
 0.617744  0.671167  0.705941  0.721242     0.474389  0.549492  0.617744
 0.705484  0.721137  0.717449  0.694357     0.616621  0.670358  0.705484
 0.718864  0.720318  0.702416  0.665

In [179]:
function generate_sample(mps_original::MPS; dx=0.1)
    mps = deepcopy(mps_original)
    s = siteinds(mps)
    xs = 0.0:dx:1.0

    x_samples = Vector{Float64}(undef, length(mps))
    for i in eachindex(mps)
        orthogonalize!(mps, i)
        ρ = prime(mps[i], s[i]) * dag(mps[i])
        # check properties
        if !isapprox(real(tr(ρ)), 1.0; atol=1E-3) @warn "Trace of RDM ρ at site $i not equal to 1 ($(abs(tr(ρ))))." end
        if !isequal(ρ.tensor, adjoint(ρ).tensor) @warn "RDM at site $i not Hermitian." end
        ρ_m = matrix(ρ)
        probs = [real(feature_map(x)' * ρ_m * feature_map(x)) for x in xs];
        probs_normed = probs ./ sum(probs)
        cdf = cumsum(probs_normed)
        r = rand()
        cdf_selected_index = findfirst(x -> x > r, cdf)
        selected_x = xs[cdf_selected_index]
        x_samples[i] = selected_x
        selected_state = feature_map(selected_x)
        site_measured_state = ITensor(selected_state, s[i])
        m = MPS(1)
        m[1] = site_measured_state
        # make into a projector
        site_projector = projector(m)
        # make into projector operator
        site_projector_operator = op(matrix(site_projector[1]), s[i])
        mps[i] *= site_projector_operator
        noprime!(mps[i])
        normalize!(mps)

    end

    return x_samples

end

generate_sample (generic function with 1 method)

In [180]:
states = slice_mps_into_label_states(loaded_mps);

Class 0 state: [1.0, 0.0]
Class 1 state: [0.0, 1.0]


In [181]:
s = siteinds(states[1]);
samp = X_train_scaled[2,:]
ps = MPS([ITensor(feature_map(samp[i]), s[i]) for i in eachindex(samp)]);

In [187]:
res = 1
for i in eachindex(states[1])
    res *= states[1][i] * conj(ps[i])
end

In [185]:
abs2(res[])/abs(inner(states[2], states[2]))

1.2854894643552516e-32

In [199]:
function forecast_sites(mps::MPS, sample::Vector, start_site::Int)
    """Assumes forward sequential interpolation for now, i.e., 
    is sample corresponds to sites 1:50, then interpolate sites 51 to 100.
    Start site is the starting point IN THE MPS (last site in sample + 1).
    Return a new mps conditioned on the sample."""
    @assert length(mps) > length(sample) "Sample is longer than MPS."
    s = siteinds(mps)
    @assert isapprox(norm(mps), 1.0; atol=1E-3) "MPS is not normalised!"

    for i in 1:(start_site-1)
        # condition each site in the mps on the sample values
        # start by getting the state corresponding to the site
        site_state = ITensor(feature_map(sample[i]), s[i])
        # construct projector, need to use 1 site mps to make one site projector 
        m = MPS(1)
        m[1] = site_state
        site_projector = projector(m)
        # turn projector into a local MPO
        site_projector_operator = op(matrix(site_projector[1]), s[i])
        orthogonalize!(mps, i)
        mps[i] *= site_projector_operator
        noprime!(mps[i])
        # normalise 
        normalize!(mps)
    end

    # now generate the remaining sites by sampling from the conditional distribution 
    x_samples = []
    for i in start_site:length(mps)
        orthogonalize!(mps, i)
        # get the rdm 
        rdm = prime(mps[i], s[i]) * dag(mps[i])
        # check properties
        if !isapprox(real(tr(rdm)), 1.0; atol=1E-3) @warn "Trace of RDM ρ at site $i not equal to 1 ($(abs(tr(rdm))))." end
        if !isequal(rdm.tensor, adjoint(rdm).tensor) @warn "RDM at site $i not Hermitian." end
        rdm_m = matrix(rdm)
        # now sample from the rdm
        norm_factor = get_normalisation_factor(rdm_m)
        #println(norm_factor)
        sampled_x, sampled_state = sample_individual_state(rdm_m, norm_factor)
        push!(x_samples, sampled_x)

        # construct projector
        sampled_state_as_ITensor = ITensor(sampled_state, s[i])
        m = MPS(1)
        m[1] = sampled_state_as_ITensor
        state_projector = projector(m)
        # make into a local MPO
        state_projector_operator = op(matrix(state_projector[1]), s[i])
        # apply to mps at site i
        mps[i] *= state_projector_operator
        # unprime indicies on updated site - indices get primed when applying MPO 
        noprime!(mps[i])
        normalize!(mps)
    end

    return x_samples
        
end

forecast_sites (generic function with 1 method)